# Clase 20: Optimización de Código en Python

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

**Profesor: Pablo Badilla**

Basados en las clases de *Nicolás Caro*



---

## Optimización de Código en Ciencia de Datos

El flujo de trabajo en ciencia de datos consta de **numerosas rutinas de carga, procesamiento y visualización**. Lo ideal es que diseñemos estas rutinas de la forma más optima posible con el fin de reducir recursos, tiempos de carga utilizados y sus costos asociados. Estas optimizaciones son abordadas por medio de algoritmos, técnicas de programación y diseño de código. 

---

## Lenguajes de Programación

El lenguaje de máquina es el conjunto de instrucciones que el hardware es capaz de interpretar y procesar.
A través de estas instrucciones podemos lograr que nuestro procesador ejecute distintos tipos de acciones muy básicas. 
Este conjuntos de lenguajes es comunmente conocido como *lenguaje de bajo nivel*

<center>
<img src='./resources/codigo_maquina.png' width=400 />
<center/>

<center>Por suerte no tenemos que si quiera pensar en esto...</center>
    
<center> 
    Fuente: <a href='https://en.wikipedia.org/wiki/Machine_code#/media/File:W65C816S_Machine_Code_Monitor.jpeg'>Wikipedia </a>
</center>
    
    
    


La idea general de un lenguaje de programación que conocemos hasta ahora (como `python`) es tener una forma *humana* de comunicarse con la máquina. Es decir, que nos permitan escribir código de forma sencilla y que sean luego convertidos al lenguaje que la máquina sea capaz de entender.

Estos lenguajes se denominan *lenguajes de alto nivel*.

> **Pregunta:** ¿Qué otros lenguajes de programación conocen y cuales son sus principales diferencias?

---

## Lenguajes Compilados vs Intepretados

Existen dos enfoques principales para convertir un código de lenguaje de alto nivel a uno de bajo nivel: que el lenguaje sea **Compilado** o **Interpretado**.

<center>
<img src='./resources/tipos_lenguajes.png' width=800/>

</center>

---

## Computación de alto Rendimiento con Python

Python es utilizado transversalmente, ya sea en la industria o en la academia. Dentro de sus cualidades se encuentra la portabilidad de código, sintaxis intuitiva, disponibilidad de herramientas y documentación. Sin embargo, al ser un lenguaje interpretado se pierden ciertas características intrínsicas de los lenguajes de bajo nivel como C, C++ y Fortran.

En esta y la próxima clase estudiaremos distintas herramientas para mejorar el rendimiento del interprete, como el uso eficiente de objetos base y la aplicación de técnicas de paralelismo y compilación utilizando tanto librerías nativas, como desarrolladas por terceros. 


> **Pregunta:** ¿Será conveniente programar siempre pensando crear código óptimo?

---

## Perfilamiento y Referenciación

 Como directriz general, se recomienda llevar el proceso de desarrollo en dos etapas:
 
1. La primera consiste en **generar código correctas, comprensibles y mantenibles**, evitando la sobre-optimización prematura de código. 

2. Como segunda etapa, se recomienda comenzar con los procesos de **optimización de código**. Esto pues, las herramientas que permiten mejorar los aspectos computacionales, interfieren en la sencillez del código, entorpeciendo los procesos de depuración y mantención. 

Una vez que las rutinas están implementadas de manera correcta, la mejor manera de enfocar los esfuerzos, pasa por **perfilar** (*profiling*) el código. Esto consiste en encontrar las zonas de código criticas en cuanto a carga computacional. La manera más directa de encontrar estas zonas, es por medio del uso de contadores de tiempo o *timers*.



### Medición del Tiempo de Ejecución

En el siguiente código usaremos la librería `time` para medir el tiempo de ejecución.

In [1]:
import time
from math import cos, sin

Definimos un rango de datos a operar 

In [2]:
x = [0.1 * i for i in range(1000)]

x[0:10]  # veamos los datos

[0.0,
 0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6000000000000001,
 0.7000000000000001,
 0.8,
 0.9]

Luego definimos la función que mediremos. Esta simplemente acumula los valores de `sin(val) + cos(val) ^ 2` en la variable `result`.

In [4]:
def func_1(a):
    result = 0
    for val in a:
        result += sin(val) + cos(val) ** 2
    return result

Ahora, estudiamos el tiempo de ejecución por medio de la función `process_time`.

In [5]:
# tiempo inicial
t0 = time.process_time()
for r in range(1000):
    func_1(x)
# tiempo final
t1 = time.process_time()

# el tiempo transcurrido es simplemente el delta entre t1 y t0
print("Tiempo transcurrido", t1 - t0)

Tiempo transcurrido 0.2376553


---

### `timeit`

En algunas ocasiones se desea medir el tiempo de ejecución para tareas sencillas, la librería estándar de Python provee el módulo `timeit`. En la práctica, una llamada de `timeit` ejecuta 1.000.000 el código y luego reporta el tiempo de ejecución promedio.

Este puede ser utilizado directamente en la consola interactiva IPython o en notebooks de Jupyter por medio del comando mágico `%timeit`. 

**Ejemplo**

Se mide el tiempo de ejecución promedio para la función coseno de NumPy.

In [6]:
import numpy as np

In [7]:
%timeit np.cos(0.5)

733 ns ± 38 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Se compara con la función del módulo `Math`

In [8]:
%timeit cos(0.5)

52.7 ns ± 1.15 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Se puede ver una gran diferencia en los tiempos de ejecución promedio. 

**Ejercicio**

1. Diferencie los comandos `%time`, `%timeit`, `%%timeit`.
2. En Pyhton existen 2 maneras para declarar listas, tuplas y diccionarios, una de ellas es la *forma funcional* y viene dada por la función de conversión de tipo de datos, por ejemplo para diccionarios es `dict()`. La otra forma es utilizar *expresiones literales* esto quiere decir declarar listas usando `[]`, tuplas usando `()` y diccionarios / conjuntos usando `{}`. Una de estas dos maneras es más rápida que la otra, utilice `%timeit` para deducir cual. 
3. ¿Qué operación es más eficiente ?

    1. `list(range())`
    2. `[*range()]`

---

### Perfilador y `prun`

Un **perfilador**  (*profiler*) es un programa que ejecuta una función y monitorea sus subfunciones, obteniendo métricas de rendimiento como el consumo de tiempo y memoria. 

Por otra parte, la referenciación (*benchmarking*) consiste en extraer zonas de código de interés para probar su rendimiento antes y después de aplicar técnicas de optimización. 

IPython provee de un perfilador de código dado por la orden `%prun`.

**Ejemplo**

1. Perfilaremos una función utilizando `%prun`. En primera instancia se define tal función `benchmark_sum` que acumula para cada `i` en `n` los valores de la siguiente forma:

Por cada $i$ en $n$:

`to_sum =` `[`${\frac{i}{2}}^n + (i - n)^{\frac{n}{3}}$ `for i in range(n)]`

`sum_ =` $\sum_{i=0}^{n}$ `to_sum`$_{i}$ 


In [12]:
def benchmark_sum(n):
    """Funcion de referencia que suma n elementos transformados
    """
    ac = []

    for i in range(n):
        to_sum = [(i // 2) ** n + (i - n) ** (n // 3) for i in range(n)]
        sum_ = sum(to_sum)
        ac.append(sum_)

    return ac

Se perfila la función con `%prun`

In [13]:
%prun benchmark_sum(500)

         1504 function calls in 1.531 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      500    1.469    0.003    1.469    0.003 <ipython-input-12-cbf893e5e842>:7(<listcomp>)
      500    0.046    0.000    0.046    0.000 {built-in method builtins.sum}
        1    0.015    0.015    1.530    1.530 <ipython-input-12-cbf893e5e842>:1(benchmark_sum)
      500    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    1.531    1.531 <string>:1(<module>)
        1    0.000    0.000    1.531    1.531 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

EL resultado corresponde las mediciones temporales de cada función involucrada en la ejecución de `benchmark_sum(500)`. En este caso, la mayoria del tiempo se utiliza en la ejecución de la compresión de listas `<listcomp>`. Esto indica que la mejor manera de optimizar el código de `benchmark_sum`pasa por optimizar tal sección del código.

---

### `lineprofiler`

Con `%lprun` es posible perfilar por linea de código, para ello es necesario instalar el módulo `line_profiler`. 

**Ejemplo**

Se carga la extensión `%lprun` y se prueba con `benchmark_sum`.

In [14]:
!pip install line_profiler

In [15]:
%load_ext line_profiler

El comando `%lprun` toma como parámetro una orden de Python y su principal argumento. Las funciones que se desean perfilar deben ser especificadas de manera explicita con la orden `-f`. En el caso de `benchmark_sum` esto se haría según el siguiente código

In [16]:
%lprun -f benchmark_sum benchmark_sum(500)

Timer unit: 1e-06 s

Total time: 1.57546 s
File: <ipython-input-12-cbf893e5e842>
Function: benchmark_sum at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def benchmark_sum(n):
     2                                               """Funcion de referencia que suma n elementos transformados
     3                                               """
     4         1          2.0      2.0      0.0      ac = []
     5                                           
     6       501        307.0      0.6      0.0      for i in range(n):
     7       500    1529765.0   3059.5     97.1          to_sum = [(i // 2) ** n + (i - n) ** (n // 3) for i in range(n)]
     8       500      44360.0     88.7      2.8          sum_ = sum(to_sum)
     9       500       1027.0      2.1      0.1          ac.append(sum_)
    10                                           
    11         1          0.0      0.0      0.0      return ac

El resultado es una tabla con el tiempo utilizado en cada linea de las funciones perfiladas, mostrando porcentajes del tiempo consumido en cada paso.

---

### Ejemplo de Benchmark

Definimos en la siguiente celda una versión optimizada de la función anterior usando `numpy` y medimos.

In [18]:
def benchmark_sum_numpy(n):
    """Funcion de referencia que suma n elementos transformados
    """
    ac = []

    for i in range(n):
        to_sum = np.array(
            [
                (i // 2) ** n + (i - n) ** (n // 3)
                for i in np.arange(n, dtype="float128")
            ],
        )
        sum_ = np.sum(to_sum)
        ac.append(sum_)

    return ac

In [19]:
%lprun -f benchmark_sum_numpy benchmark_sum_numpy(500)

Timer unit: 1e-06 s

Total time: 0.60456 s
File: <ipython-input-18-1c6018a885e0>
Function: benchmark_sum_numpy at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def benchmark_sum_numpy(n):
     2                                               """Funcion de referencia que suma n elementos transformados
     3                                               """
     4         1          1.0      1.0      0.0      ac = []
     5                                           
     6       501        341.0      0.7      0.1      for i in range(n):
     7      1000      22150.0     22.1      3.7          to_sum = np.array(
     8      1000     550867.0    550.9     91.1              [
     9                                                           (i // 2) ** n + (i - n) ** (n // 3)
    10       500      14349.0     28.7      2.4                  for i in np.arange(n, dtype="float128")
    11                                  

**Ejercicio**

1. Los perfiladores `%prun` y `%lprun` tienen en común los argumentos -D, -T y -r utilice el parámetro `?` para investigar estas opciones.

---

### `memory_profiler`

Es posible perfilar uso de memoria, para ello existen los comandos mágicos `%memit` y `%mprun`. Para utilizarlos es necesario instalar el módulo `memory_profiler` y cargarlo mediante

```python
%load_ext memory_profiler
```

**Ejemplo**

In [20]:
!pip install memory_profiler

In [21]:
%load_ext memory_profiler

En primera instancia perfilamos utilizando la linea mágica `%memit`, la cual es equivalente a `timeit` pero ofrece medidas sobre el uso de memoria. 

In [22]:
%memit benchmark_sum(500)

peak memory: 82.67 MiB, increment: 0.59 MiB


In [23]:
%memit benchmark_sum_numpy(500)

peak memory: 82.47 MiB, increment: 0.03 MiB


Se puede observar un uso de memoria en torno a 90 MB.

De manera análoga a `%prun` la librería `memory_profiler` permite utilizar `%mprun`, con la cual se pueden obtener descripciones linea a linea del uso de memoria. El uso de este comando es un poco más restrictivo, pues solo permite medir funciones definidas en módulos (no dentro de un notebook). Para ello, se crea el módulo `memory_demo`. La manera sencilla de hacer esto, es mediante el comando mágico `%%file` este permite crear archivos en el directorio de trabajo actual, utilizando el código dentro de una celda de jupyter. 

Se procede a generar el módulo que contiene el código de `benchmark_sum` utilizando el comando `%%file`.




In [24]:
%%file bench_module.py


def benchmark_sum(n):
    """Funcion de referencia que suma n elementos transformados
    """
    ac = []

    for i in range(n):
        to_sum = [(i // 2) ** n + (i - n) ** (n // 3) for i in range(n)]
        sum_ = sum(to_sum)
        ac.append(sum_)

    return ac

Overwriting bench_module.py


a continuación, se importa el módulo creado y se perfuila su memoria mediante:

In [25]:
from bench_module import benchmark_sum

%mprun -f benchmark_sum benchmark_sum(500)

Filename: /home/pablo/MDS7202/clases/20-Optimización-Código/bench_module.py

Line #    Mem usage    Increment  Occurences   Line Contents
     3     82.7 MiB     82.7 MiB           1   def benchmark_sum(n):
     4                                             """Funcion de referencia que suma n elementos transformados
     5                                             """
     6     82.7 MiB      0.0 MiB           1       ac = []
     7                                         
     8     83.0 MiB      0.0 MiB         501       for i in range(n):
     9     83.0 MiB      0.3 MiB      251500           to_sum = [(i // 2) ** n + (i - n) ** (n // 3) for i in range(n)]
    10     83.0 MiB      0.0 MiB         500           sum_ = sum(to_sum)
    11     83.0 MiB      0.0 MiB         500           ac.append(sum_)
    12                                         
    13     83.0 MiB      0.0 MiB           1       return ac

El resultado se muestra en pantalla, obteniendo detalles linea a linea.

---

## Optimización del Código Nativo

Una de las manera más eficientes de mejorar el rendimiento de aplicaciones es por medio del uso de algoritmos más eficientes en conjunción de estructuras de datos mejor diseñadas. A continuación, se estudian los algoritmos y estructuras de datos presentes de manera nativa en Python que permiten acelerar ciertas rutinas. 

En términos generales, los algoritmos pueden ser clasificados según su **complejidad computacional**. Esta clasificación se expresa según la notación de **O-grande, que corresponde a una cota superior de las operaciones requeridas para ejecutar una tarea.**

Ejemplos: 

- $O(n)$ requiere ejecutar una operación por cada elemento de un arreglo.
- $O(1)$ requiero para acceder a cierta llave de un diccionario. Noten que esto es totalmente independiente del número de datos que tenga el diccionario.

**Ejemplo**

Se genera un lista, a cada uno de sus elementos se le realiza una operación básica.

In [26]:
lista = list(range(10))

for i in range(len(lista)):
    lista[i] += 100

In [27]:
lista

[100, 101, 102, 103, 104, 105, 106, 107, 108, 109]

En este algoritmo, la operación `lista[i]+=100` es repetida tantas veces como elementos hay en `lista`, que corresponde al tamaño de los datos de entrada. Al observar la que las operaciones realizadas por este algoritmo son proporcionales a la cantidad de elementos de `lista`, se puede decir que su tiempo de ejecución es $O(N)$ donde `N = len(lista)`. 

### Optimización de Operaciones con Listas

Las listas de Python son **colecciones ordenadas de elementos**, estás se encuentran clasificadas como *arreglos*, que a la vez corresponden a una estructura de datos caracterizada por contener elementos contiguos en bloques de memoria, cada uno de los cuales contienen una referencia a un objeto de Python. La ventaja de las listas recae en la facilidad que entregan par acceder, modificar y agregar elementos. 

#### Acceder y Modificar

Dado que acceder y modificar elementos de una lista corresponde a acceder a espacios de memoria que a priori no dependen de la longitud de la lista, se dice que estas operaciones tienen complejidad $O(1)$. 


```python
[0, 1, 2, 3]
#   ↑
# acceder y modificar cualquier elemento es tiempo constante O(1)    
```

#### Agregar al Final

Por otra parte, para agregar un elemento a una lista por medio de `.append()`, puede requerirse re-ubicar la memoria del arreglo asociado, operación que toma un tiempo de $O(N)$. Sin embargo, tal operación es muy poco frecuente, pues por lo general se tiene acceso a bloques de memoria contiguos, por tal motivo, se dice que la operación `.append()` tiene un tiempo esperado de ejecución de $O(1)$. 

```python
[0, 1, 2, 3]

#si ejemcutamos append(4)

[0, 1, 2, 3, 4]
#            ↑
# agregar por lo general es O(1)    
```


#### Agregar (`insert`) /Eliminar al inicio

Para agregar o eliminar datos al inicio de un arreglo, se requiere hacer una traslación (o *shift*) de los demás elementos por lo que tal operación toma un tiempo de $O(N)$. Para agregar o remover elementos de un arreglo en una posición distinta a la última, se opera de manera análoga. 

```python
[0, 1, 2, 3]

#si ejemcutamos insert(0, -1) tendremos que desplazar todo el arreglo a la derecha.

[-1, 0, 1, 2, 3]
# ↑, →, →, →, →
# agregar al inicio implicará desplazar todo. Es decir, O(n)
```


```python
[0, 1, 2, 3]

#si ejemcutamos pop(0) tendremos que desplazar todo el arreglo a la izquierda.

[ 1, 2, 3]
# ←, ←, ←
# eliminar al inicio implicará desplazar todo. Es decir, O(n)
```



**Ejemplo**

Se definen listas para estudiar la complejidad de ciertos métodos empíricamente. Se definen los parámetros

In [28]:
n_0, n_1, n_2 = (int(10e5), int(5 * 10e5), int(10e6))

In [29]:
n_0

1000000

In [30]:
n_1

5000000

In [31]:
n_2

10000000

Se generan una funciones de referencia

In [36]:
def copy_objs(obj_0, obj_1, obj_2):
    """Abstraccion auxiliar para copiar elementos."""
    return (obj_0.copy(), obj_1.copy(), obj_2.copy())


def bench_pop(l_0, l_1, l_2, arg=-1):
    """Funcion de referncia para eliminacion de elementos."""

    l_0.pop(arg)
    l_1.pop(arg)
    l_2.pop(arg)


def bench_append(l_0, l_1, l_2, arg=1):
    """Funcion de referencia para insertar 1 con append."""

    l_0.append(arg)
    l_1.append(arg)
    l_2.append(arg)


def bench_insert(l_0, l_1, l_2, args=(0, 1)):
    """Funcion de referncia para insertar 1 con insert."""

    l_0.insert(*args)
    l_1.insert(*args)
    l_2.insert(*args)

Se construye el test 

In [34]:
lista_0, lista_1, lista_2 = (list(range(n_0)), list(range(n_1)), list(range(n_2)))

In [35]:
len(lista_0)

1000000

In [37]:
len(lista_1)

5000000

In [38]:
len(lista_2)

10000000

#### Eliminar el ultimo elemento

In [39]:
l_0, l_1, l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo constante
%lprun -f bench_pop bench_pop(l_0,l_1,l_2)

Timer unit: 1e-06 s

Total time: 5e-06 s
File: <ipython-input-36-d1144422c01d>
Function: bench_pop at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def bench_pop(l_0, l_1, l_2, arg=-1):
     7                                               """Funcion de referncia para eliminacion de elementos."""
     8                                           
     9         1          2.0      2.0     40.0      l_0.pop(arg)
    10         1          2.0      2.0     40.0      l_1.pop(arg)
    11         1          1.0      1.0     20.0      l_2.pop(arg)

#### Eliminar el primer elemento

In [40]:
l_0, l_1, l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo lineal
%lprun -f bench_pop bench_pop(l_0,l_1,l_2,0)

Timer unit: 1e-06 s

Total time: 0.010725 s
File: <ipython-input-36-d1144422c01d>
Function: bench_pop at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def bench_pop(l_0, l_1, l_2, arg=-1):
     7                                               """Funcion de referncia para eliminacion de elementos."""
     8                                           
     9         1        608.0    608.0      5.7      l_0.pop(arg)
    10         1       3060.0   3060.0     28.5      l_1.pop(arg)
    11         1       7057.0   7057.0     65.8      l_2.pop(arg)

#### Insertar 1 en la ultima posicion

In [41]:
l_0, l_1, l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo constante (casi seguramente)
%lprun -f  bench_append bench_append(l_0,l_1,l_2)

Timer unit: 1e-06 s

Total time: 0.002565 s
File: <ipython-input-36-d1144422c01d>
Function: bench_append at line 14

Line #      Hits         Time  Per Hit   % Time  Line Contents
    14                                           def bench_append(l_0, l_1, l_2, arg=1):
    15                                               """Funcion de referencia para insertar 1 con append."""
    16                                           
    17         1       2236.0   2236.0     87.2      l_0.append(arg)
    18         1        325.0    325.0     12.7      l_1.append(arg)
    19         1          4.0      4.0      0.2      l_2.append(arg)

#### Insertar 1 en la primera posicion

In [42]:
l_0, l_1, l_2 = copy_objs(lista_0, lista_1, lista_2)

# Se observa un tiempo lineal
%lprun -f bench_insert bench_insert(l_0,l_1,l_2)

Timer unit: 1e-06 s

Total time: 0.02078 s
File: <ipython-input-36-d1144422c01d>
Function: bench_insert at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
    22                                           def bench_insert(l_0, l_1, l_2, args=(0, 1)):
    23                                               """Funcion de referncia para insertar 1 con insert."""
    24                                           
    25         1        883.0    883.0      4.2      l_0.insert(*args)
    26         1       3646.0   3646.0     17.5      l_1.insert(*args)
    27         1      16251.0  16251.0     78.2      l_2.insert(*args)

---

### Double-Ended Queue: `deque` 


<img src='./resources/deque.png'/>

<center>Fuente: <a href='https://medium.com/@rasmussen.matias/fun-with-deques-in-python-31942bcb6321'>Matias Rasmussen en Medium </a>
    
</center>

Para efectuar inserciones de manera eficiente (siempre en tiempo constante) Se puede utilizar la estructura de datos `deque` del módulo `collections`. Estas estructuras se comportan como listas, están diseñadas para acelerar la inserción de objetos y añaden los métodos `.popleft` y `.appendleft`. 




**Ejemplo**

Se compara `.popleft` en *deques* con `.pop(0)` en listas.

In [43]:
from collections import deque


def bench_pop_left(d_0, d_1, d_2):
    """Funcion de referncia para eliminacion de elementos."""

    d_0.popleft()
    d_1.popleft()
    d_2.popleft()


def bench_append_left(d_0, d_1, d_2, val=1):
    """Funcion de referncia para insertar 1 con insert."""

    d_0.appendleft(val)
    d_1.appendleft(val)
    d_2.appendleft(val)

Se definen los objetos sobre los que se trabajará

In [47]:
deque_0, deque_1, deque_2 = tuple(map(deque, [lista_1, lista_1, lista_2]))

In [48]:
d_0, d_1, d_2 = copy_objs(deque_0, deque_1, deque_2)

# Se observa un tiempo constante
%lprun -f bench_pop_left bench_pop_left(d_0, d_1, d_2)

Timer unit: 1e-06 s

Total time: 5e-06 s
File: <ipython-input-43-60013af0a2d7>
Function: bench_pop_left at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def bench_pop_left(d_0, d_1, d_2):
     5                                               """Funcion de referncia para eliminacion de elementos."""
     6                                           
     7         1          3.0      3.0     60.0      d_0.popleft()
     8         1          1.0      1.0     20.0      d_1.popleft()
     9         1          1.0      1.0     20.0      d_2.popleft()

In [49]:
%%timeit
d_0, d_1, d_2 = copy_objs(deque_0, deque_1, deque_2)

170 ms ± 7.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Copiar objetos de tipo `deque` tiene una carga de aproximadamente 190 ms

In [50]:
%%timeit
d_0, d_1, d_2 = copy_objs(deque_0, deque_1, deque_2)
bench_pop_left(d_0, d_1, d_2)

236 ms ± 7.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Por su parte, aplicar el benchmark `bench_pop_left` tarda en promedio 191 - 190 ms = 1 ms.

En cuanto a las listas, la operación de copiar lleva unos 112 ms en promedio

In [51]:
%%timeit
l_0, l_1, l_2 = copy_objs(lista_0, lista_1, lista_2)

146 ms ± 6.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Aplicar el benchmark `bench_pop` en listas lleva un tiempo promedio de 120 ms - 112 ms = 8ms. Por lo que se aprecia un aumento en el rendimiento. Cabe señalar que tal aumento se ve sujeto a una carga mayor en el proceso de copia de objetos, por tal motivo, vale la pena evitar la copia de objetos tipo `deque` y utilizarlos para acceder a lista con una gran cantidad de elementos.

In [52]:
%%timeit
l_0, l_1, l_2 = copy_objs(lista_0, lista_1, lista_2)
bench_pop(lista_0, lista_1, lista_2, 0)

140 ms ± 19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Ejercicios**

1. Repita el proceso de comparación para `.appendleft` en deques  e `.insert()` en listas. 

2. Cual es la complejidad computacional de acceder a:
    1. Primer elemento de un deque / lista
    2. Ultimo elemento de un deque / lista
    3. Un elemento distinto del último o el primero (ej: el elemento de la mitad de un arreglo).

3. El módulo `bisect` permite hacer búsquedas rápidas en arreglos ordenados. la función `bisect.bisect` permite encontrar el índice en cual insertar un elemento, manteniendo el orden del arreglo operado. 
    1. Genere la lista ordenada 'ordered_list'  de números entre 0 y 10.
    2. Elimine el cuarto elemento de la lista, guarde su valor en la variable `dropped`. Se debe hacer en una linea de código. 
    3. Importe el módulo `bisect` y utilice el comando `bisect.bisect(ordered_list,dropped)`. ¿Qué significa el valor retornado por la función?¿qué operación se efectúa por medio del comando recién aplicado?
    4. Haga un código de referencia para comparar las funciones `list.index()` y `bisect.bisect()` por medio de perfilamiento temporal. Para comprobar sus resultados utilice el hecho de que  el tiempo de ejecución para `bisect.bisect` es de $O(\log(N)$, mientras que el de `list.index()` es de $O(N)$. 
    5. Estudie la función `bisect.bisect_left`. 
    

---

### Optimización de Operaciones con Diccionarios

La gran flexibilidad de los diccionarios los hacen un objeto central en el uso de Python. Estos son implementaciones de *hash maps*, es decir, son estructuras de datos construidas por medio de asociaciones *llave - valor*, donde a cada llave, se asigna un índice especifico, de tal manera que el valor de tal índice puede ser ordenado en un arreglo. Por tal motivo, los diccionarios son altamente eficientes en procesos de eliminación, acceso e inserción teniendo un tiempo promedio de ejecución de $O(1)$. 


<center>
    <img src='./resources/hashmap.png' />
    Fuente: <a href='https://techmastertutorial.in/java-collection-internal-hashmap.html'>Java HashMap en techmastertutorial.in</s>
              
              
</center>

Para acceder a los índices dados por el *hash map* se puede utilizar la función `hash` de Python, esta opera sobre distintos tipos de datos.




**Ejemplo**

Se aplica `hash` a diferentes objetos

In [53]:
print("hash string: ", hash("MDS7202"))
print("hash int:", hash(1234))
print("hash tuple", hash(("a", "b", "c")))

hash string:  6566211537489362102
hash int: 1234
hash tuple 6559510708141207297


Un objeto puede ser operado por `hash` (*hashable object*) si tiene un método `__hash__` y puede ser comparado por medio de `__eq__` por ejemplo. Si un objeto es *hashable* significa que puede ser utilizado como llave de un diccionario, en general, todos los objetos inmutables de Python son *hashables* mientras que las listas y diccionarios, por ser inmutables, no lo son.  



#### `defaultdict`

In [54]:
from collections import defaultdict

Los objetos `defaultdict` poseen todas las funcionalidades de un diccionario pero añaden el método `.__missing__()` con el cual se proveen valores por defecto, los cuales se asignan a una nueva llave, es decir, permiten inicializar diccionarios entregando solo el valor de la llave (y no su valor asociado), pues a cada llave nueva, se asigna un valor por defecto de manera automática. 


In [55]:
# list es el valor por defecto que tendrá cada llave del diccionario,
# aunque esta no se haya definido.
d = defaultdict(list)
d

defaultdict(list, {})

In [56]:
"hola" in d

False

In [58]:
hash("hola")

20342891857556649

In [57]:
# esto inicializa la variable hola.
d["hola"]

[]

In [60]:
d["hola2"]

[]

In [ ]:
"hola" in d

In [61]:
# vean como se agregó la llave 'hola' junto a una lista vacía [] al diccionario
d

defaultdict(list, {'hola': [], 'hola2': []})

El comportamiento anterior, como vimos anteriormente, no es posible en un diccionario común y corriente:

In [62]:
d2 = {}
d2["hola"]

KeyError: 'hola'

Para perfilar los `defaultdict`, creamos una lista de tuplas (llave-valor) `to_group` las cuales las queremos agrupar y luego ejecutamos %%timeit sobre ellas.

In [64]:
to_group = [
    ("a", 1),
    ("b", 2),
    ("c", 3),
    ("b", 4),
    ("d", 1),
]

Esto es lo que quisieramos lograr:

```python
salida_esperada = [('a', [1]), 
                   ('b', [2, 4]), 
                   ('c', [3]), 
                   ('d', [1])]
```


In [68]:
D = defaultdict(list)

for k, v in to_group:
    D[k].append(v)

D

defaultdict(list, {'a': [1], 'b': [2, 4], 'c': [3], 'd': [1]})

In [65]:
%%timeit # timeit con %% mide toda la celda.
D = defaultdict(list)

for k, v in to_group:
    D[k].append(v)

sorted(D.items())

1.75 µs ± 70.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


El código anterior genera las llaves `k`, que por defecto poseen una lista vacía asociada, a cada lista vacía agregan por medio de `append` (tiempo de ejecución constante) el elemento inspeccionado `v`. 

Ahora, comprobemos la eficiencia del diccionario con respecto a una implementación con listas.
Para esto, implementamos la misma funcionalidad, pero con  `for` y `append`  y luego perfilamos utilizando `%%timeit`:

In [69]:
%%timeit
L = []
for elem in to_group:
    if len(L) == 0:
        L.append((elem[0], [elem[1]]))
    else:
        c = 0
        for l in L:
            if l[0] == elem[0]:
                l[1].append(elem[1])
                c = 1
        if c == 0:
            L.append((elem[0], [elem[1]]))

L

1.93 µs ± 86.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Con lo anterior apreciamos una ganancia en eficiencia, que tambien se traduce en simpleza.

**Ejercicios**

Los objetos `defaultdict` permiten además aumentar la eficiencia al momento de contar elementos de un arreglo. Para ver esto, implemente una función que:

1. Reciba una *iterable* como argumento.
2. Inicalice un objeto `defaultdict` con tipo de dato `int`. ¿Qué valor se asocia por defecto?.
3. Recorra cada elemento del iterable, registrando su número de ocurrencias en una llave del objeto `defaultdict` antes inicializado.
    

---

### `Counter`

El módulo `collections` permite implementar el procedimiento del ejercicio 2 anterior por medio de la clase `Counter`


In [70]:
from collections import Counter

import numpy as np

lista = np.random.randint(0, 10, size=100)
counts = Counter(lista)

sorted(counts.items())

[(0, 7),
 (1, 15),
 (2, 18),
 (3, 11),
 (4, 5),
 (5, 13),
 (6, 7),
 (7, 11),
 (8, 5),
 (9, 8)]

**Obs**: El método de counteo por medio de la clase `Counter` tiene un tiempo de ejecución de $O(N)$.

Otra ventaja de los diccionarios es que permiten buscar palabras de manera rápida en una *lista de documentos*. 

Por ejemplo, *El hombre imaginario*. (aquí asumimos que cada elemento del arreglo es un documento por separado).

In [71]:
with open("./resources/text.txt", "r") as file:
    lines = file.readlines()
    lines = [l.rstrip(" \n") for l in lines]

lines

['El hombre imaginario',
 'vive en una mansión imaginaria',
 'rodeada de árboles imaginarios',
 'a la orilla de un río imaginario',
 '',
 'De los muros que son imaginarios',
 'penden antiguos cuadros imaginarios',
 'irreparables grietas imaginarias',
 'que representan hechos imaginarios',
 'ocurridos en mundos imaginarios',
 'en lugares y tiempos imaginarios',
 '',
 'Todas las tardes imaginarias',
 'sube las escaleras imaginarias',
 'y se asoma al balcón imaginario',
 'a mirar el paisaje imaginario',
 'que consiste en un valle imaginario',
 'circundado de cerros imaginarios',
 '',
 'Sombras imaginarias',
 'vienen por el camino imaginario',
 'entonando canciones imaginarias',
 'a la muerte del sol imaginario',
 '',
 'Y en las noches de luna imaginaria',
 'sueña con la mujer imaginaria',
 'que le brindó su amor imaginario',
 'vuelve a sentir ese mismo dolor',
 'ese mismo placer imaginario',
 'y vuelve a palpitar',
 'el corazón del hombre imaginario']

Supongamos que queremos buscar la palabra `'imaginario'` en cada documento. Es posible hacer esto recorriendo todo cada vez que ejecutamos la búsqueda según el siguiente código.

In [72]:
to_search = "imaginario"

found = [line for line in lines if to_search in line]
found

['El hombre imaginario',
 'rodeada de árboles imaginarios',
 'a la orilla de un río imaginario',
 'De los muros que son imaginarios',
 'penden antiguos cuadros imaginarios',
 'que representan hechos imaginarios',
 'ocurridos en mundos imaginarios',
 'en lugares y tiempos imaginarios',
 'y se asoma al balcón imaginario',
 'a mirar el paisaje imaginario',
 'que consiste en un valle imaginario',
 'circundado de cerros imaginarios',
 'vienen por el camino imaginario',
 'a la muerte del sol imaginario',
 'que le brindó su amor imaginario',
 'ese mismo placer imaginario',
 'el corazón del hombre imaginario']

In [73]:
%timeit found = [line for line in lines if to_search in line]

1.9 µs ± 33.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Se debe considerar que el tiempo de ejecución asociado a consultar por una palabra es $O(N)$. Para mejorar esto, se puede construir un diccionario, donde a cada palabra se asocie un índice, donde este último corresponde al la linea (o documento si se prefiere) al que pertenece. Esto se puede hacer mediante el siguiente código

In [74]:
index = defaultdict(list)

for i, line in enumerate(lines):

    for word in line.split():
        index[word].append(i)

In [75]:
index

defaultdict(list,
            {'El': [0],
             'hombre': [0, 30],
             'imaginario': [0, 3, 14, 15, 16, 20, 22, 26, 28, 30],
             'vive': [1],
             'en': [1, 9, 10, 16, 24],
             'una': [1],
             'mansión': [1],
             'imaginaria': [1, 24, 25],
             'rodeada': [2],
             'de': [2, 3, 17, 24],
             'árboles': [2],
             'imaginarios': [2, 5, 6, 8, 9, 10, 17],
             'a': [3, 15, 22, 27, 29],
             'la': [3, 22, 25],
             'orilla': [3],
             'un': [3, 16],
             'río': [3],
             'De': [5],
             'los': [5],
             'muros': [5],
             'que': [5, 8, 16, 26],
             'son': [5],
             'penden': [6],
             'antiguos': [6],
             'cuadros': [6],
             'irreparables': [7],
             'grietas': [7],
             'imaginarias': [7, 12, 13, 19, 21],
             'representan': [8],
             'hechos': [8],
     

En el diccionario generado, hacer búsquedas es de orden $O(1)$, luego para la misma consulta antes hecha se tiene

In [76]:
%%timeit
res = index[to_search]
[lines[i] for i in res]

697 ns ± 18.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Es decir, un aumento substancial de rendimiento. Cabe mencionar, que este procedimiento solo tiene sentido si se busca hacer una cantidad alta de consultas sobre un arreglo de lineas/documentos, esto pues, el tiempo de preprocesamiento para generar el indexado por diccionario puede ser muy alto.

---

### Optimización de Operaciones con Conjuntos

A diferencia de las listas, los conjuntos son colecciones no ordenadas, donde cada elemento debe ser único. La implementación de conjuntos en Python sigue la misma lógica de los diccionarios en cuanto ambos utilizan funciones *hash*. Por tal motivo, en conjuntos se tienen operaciones rápidas para añadir, eliminar y acceder a elementos. (Del orden $O(1)$).

**Ejercicio**

Los tiempos de ejecución para los métodos `A.union(B)`, `A.intersection(B)` y `A.difference(B)` son $O(a+b)$, $O(\min(a,b))$ y $O(a)$, donde $a = |A|$ y $b = |B|$. 

1. Construya una función de referencia para cada método y utilice un perfilamiento adecuado para comprobar la afirmación anterior de manera empírica. 

Se puede hacer uso de conjuntos para efectuar consultas rápidas.

**Ejemplo**

Se utiliza el objeto `index` creado para hacer búsquedas sobre texto. Se consulta sobre los documentos donde las palabras 'imaginario' y 'vive' ocurren simultáneamente, se obtiene un estimado del tiempo de ejecución.

In [ ]:
to_search = ["imaginaria", "vive"]

In [ ]:
res_0 = [lines[i] for i in index[to_search[0]]]
res_1 = [lines[i] for i in index[to_search[1]]]

[r for r in res_1 if r in res_0]

In [ ]:
%%timeit

res_0 = [lines[i] for i in index[to_search[0]]]
res_1 = [lines[i] for i in index[to_search[1]]]

[r for r in res_1 if r in res_0]

Se puede modificar la función de indexación para que opere sobre conjuntos.

In [ ]:
index_set = defaultdict(set)

for i, line in enumerate(lines):

    for word in line.split():
        index_set[word].add(i)

luego se hace la misma búsqueda por medio de

In [ ]:
%%timeit
index_set[to_search[0]].intersection(index_set[to_search[1]])

In [ ]:
index_set

En este apartado se ve un incremento substancial.

---

### Optimización con uso de Memoización

También se puede mejorar el rendimiento de aplicaciones por medio de un uso eficiente de la memoria, una de las ideas tras esta premisa es la de guardar los resultados de operaciones intensivas en un espacio de memoria llamado *cache*, este espacio puede estar ubicado en memoria (RAM), disco o almacenada de manera remota. El acto de guardar resultados en memoria para luego utilizarlos de manera directa se denomina **memoización**. y es una forma de *chaching* o uso de memorias *cache*.

Python ofrece el decorador `@lru_cache` accesible desde la librería base `functools`. Este decorador puede ser utilizado de manera sencilla para guardar resultados en memoria y luego accederlos. 

**Ejemplo**

Se utiliza el decorador `@lru_cache` sobre una función sencilla. En primera instancia se importa el módulo necesario.

In [ ]:
from functools import lru_cache

Se define la función a memoizar y se aplica el decorador

In [ ]:
@lru_cache
def simple_func(x, y):
    """Funcion de prueba para memoizar."""

    print("Obteniendo el resultado...")

    return x ** y + y ** x

Para comprobar el funcionamiento del decorador se llama la función dos veces sobre el mismo argumento.

In [ ]:
args = (2, 5)
simple_func(*args)

Se repite el procedimiento 

In [ ]:
simple_func(*args)

En este último caso se observa que el resultado es obtenido directamente desde la memoria. 

**Ejercicios**

`@lru_cache` es un decorador que acepta argumentos de entrada, permitiendo el uso del `max_size`. Con este parámetro se especifica el tamaño máximo de memoria para el cache asociado a la función. 

1. Decore la función anterior indicando como parámetro `max_size = 8`.

2. ¿Qué ocurre cuando se llena el tamaño maximo y se realizan más cálculos? *Hint*: lru significa *least recently used*.

3. Acceda a la información del *cache* por medio del método `.cache_info()` de la función decorada. ¿Qué significa *hit* y *miss* en este contexto?


Un ejemplo más avanzado es el de memoizar funciones recursivas

**Ejemplo**

Se trabaja con la función recursiva `factorial` almacenando sus resultados en *cache*.


```python

(factorial(5))
(5 * factorial(4))
(5 * (4 *factorial(3)))
(5 * (4 * (3 *factorial(2))))
(5 * (4 * (3 * (2 * factorial(1)))))
(5 * (4 * (3 * (2 * 1))))
(5 * (4 * (3 * 2)))
(5 * (4 * 6))
(5 * 24)
(120)

```

In [ ]:
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n - 1)

Se mide el tiempo de ejecución para `n=1000`.

In [ ]:
%%timeit
factorial(1000)

Se memoiza la función y se repite el experimento

In [ ]:
@lru_cache(maxsize=1000)
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n - 1)

In [ ]:
%%timeit
factorial(1000)

Con lo que se comprueba la eficiencia del método. 

**Ejercicios**

1. Programe secuencia de Fibonacci, perfile su consumo de tiempo y luego compare con una versión memoizada.

2. Instale el módulo `joblib`. Este módulo permite guardar resultados en disco por medio del objeto `Memory`. Utilice el decorador `@memory.cache` para memoizar as funciones anteriores (factorial y fibonacci). Compare los tiempos de ejecución al guardar los resultados en disco vs ram.

**Obs**: La ventaja de utilizar técnicas de *caching* tienen un costo, este radica en aumentar el consumo de memoria, si esta memoria esta localizada en disco, el acceso puede ser muy lento y el rendimiento puede decaer drásticamente. Antes de usar este tipo de estrategias, se recomienda estudiar la factibilidad, teniendo en cuenta las politicas de almacenamiento y acceso de los resultados y su relación con el rendimiento del programa que se desea implementar.

### Optimización con uso de Compresiones y Generadores

Las compresiones de lista están altamente optimizadas en Python y por tanto puede ser utilizadas para reemplazar ciclos `for` en ciertas circunstancias. 

**Ejemplo**

Es posible ganar un mayor rendimiento al utilizar comprensión de listas en vez del ciclo `for` en el siguiente código

In [ ]:
n = int(10e4)


def for_loop(n=n):
    res = []
    for i in range(n):
        res.append(i * (i + 1) - i ** 2)

    return sum(res) / n

In [ ]:
for_loop(n)

Se mide su consumo de tiempo por medio de `%%timeit`

In [ ]:
%%timeit
for_loop()

Se implementa la misma función haciendo uso de compresión de listas y de diccionarios.

In [ ]:
def list_comp(n=n):
    return sum([i * (i + 1) - i ** 2 for i in range(n)]) / n


def dic_comp(n=n):
    return sum({i: i * (i + 1) - i ** 2 for i in range(n)}) / n

In [ ]:
%%timeit
list_comp()

In [ ]:
%%timeit
dic_comp()

En el caso de compresión de listas vemos una mejoría, por su parte, en compresión de diccionarios, podemos esperar que operaciones de reducción sean más lentas pues se hace uso de llaves. 

En términos de memoria, cada compresión de listas (o diccionarios) ocupa un nuevo espacio, lo cual aumenta el uso de memoria. Para atacar este problema, se puede hacer uso de **generadores**. 




#### Generadores

Un generador es un iterable que guarda que posee memoria solo de su estado actual y una regla de cambio para el estado siguiente. 

##### `map`

Ejemplo: Ya hemos usado bastante la función `map`, la cual, es en verdad un generador.

In [ ]:
m = map(lambda x: x * 10, range(10))
m

In [ ]:
next(m)

In [ ]:
next(m)

In [ ]:
next(m)

In [ ]:
m

##### `filter`

Por otro lado, también hemos usado `filter`, el cual, cumple la misma propiedad que vimos en map

In [ ]:
f = filter(lambda x: x % 2 == 0, range(10))
f

In [ ]:
next(f)

In [ ]:
next(f)

In [ ]:
next(f)

In [ ]:
next(f)

In [ ]:
next(f)

Noten que cuando ya no quedan más elementos, se lanza una excepción.

In [ ]:
next(f)

**Ejemplo**

La función `map` toma como argumentos una función y un iterator, el resultado de su aplicación es un generador. Para estudiar el comportamiento de este tipo de objetos se construyen dos funciones y se perfilan ...



In [ ]:
# %%file bench_module.py -a


def list_comp_list(n=int(10e6)):
    """Concatena operaciones sobre comprensiones de lista"""

    l_1 = [i ** 2 for i in range(n) if i % 2 == 0]
    l_2 = [i * (i - 1) for i in l_1]

    l_3 = [i // 3 for i in l_2]

    return max(l_3) / n ** 2

se construye la misma función utilizando generadores `map`

In [ ]:
# %%file bench_module.py -a


def list_comp_map(n=int(10e6)):
    """Concatena operaciones sobre comprensiones de lista"""

    l_1 = map(lambda i: i ** 2, [i for i in range(n) if i % 2 == 0])
    l_2 = map(lambda i: i * (i - 1), l_1)

    l_3 = map(lambda i: i // 3, l_2)

    return max(l_3) / n ** 2

se perfila la memoria utilizada por estas funciones

In [ ]:
%memit list_comp_list()

In [ ]:
%memit list_comp_map()

**Ejercicios**

Se pueden entender los objetos generadores con las estructuras ya vistas. El objetivo de estos ejercicios es revisar ciertos aspectos de interés al usar generadores.

1. Se puede definir un generador por medio de *compresnsión de generadores* este tipo de comprensión sigue la misma sintaxis que una compresión usua (listas o diccionarios) solo que se utilizan paréntesis normales (como una '*compresion de tuplas*'). Defina un generador por medio de *compresión de generadores*.

2. Se pueden definir generadores a partir de funciones, en este caso, el comando `return` debe ser sustituido por `yield`. Este comando permite que el generador definido sea un iterable, el cual solo guarda su estado actual de ejecución y posee una *regla de transición* dada por el cuerpo de la función. Implemente un generador de secuencias inifinitas. Para ello:
    1. Defina una función `infinite_seq`, que no tiene argumentos de entrada. 
    2. Defina un acumulador de suma por medio de la variable `sum = 0`.
    3. Defina un bloque `while True` dentro de este inserte las ordenes `yield sum` y en la linea siguiente `sum +=1`. 
    4. Itere sobre su generador utilizando el método `next`. Interprete la función de `yield` en el bloque anterior. 